In [5]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader("test.txt")
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
docs = text_splitter.split_documents(documents)

Created a chunk of size 1012, which is longer than the specified 1000


In [6]:
docs

[Document(metadata={'source': 'test.txt'}, page_content='Do you remember the last time you wrote an essay by yourself? When you wrote an article? Completed an academic assignment without any external help? And for computer science students: When did you last open and read a book? Have you noticed how often you find yourself chatting with ChatGPT or other AI tools, even for the most basic questions? This method of obtaining information, bypassing the hassle of sifting through multiple web pages, PDFs, books, and references, has undoubtedly streamlined our lives. However, it’s crucial to recognize that it may be dulling our brains day by day. Have we forgotten the thrill we get when we find a piece of information that were weren’t even looking for, the joy of finding something useless at that moment but that strikes our brain and makes us ponder a bit about this vast universe, the joy of looking for clues hidden in multiple sources. Let’s delve deeper into what we’re calling “The ChatGPT

In [8]:
embeddings = OllamaEmbeddings(model ="gemma2:2b")
db = FAISS.from_documents(docs, embeddings)
db

In [10]:
query = "How is our dependency growing in AI tools"
docs = db.similarity_search(query)

docs[0].page_content

'One of the most significant impacts of the ChatGPT Effect is the potential decline in critical thinking skills. When we depend on AI to think for us, we lose the ability to evaluate information critically. AI tools provide answers, but they don’t teach us how to question, doubt, or verify. This lack of critical engagement can make us more susceptible to misinformation and reduce our ability to form well-rounded opinions.\n\nCreativity thrives on challenges, mistakes, and the journey of discovery. By outsourcing our creative tasks to AI, we risk losing the ability to think outside the box. Writing an essay, composing music, or solving a complex problem involves a series of trial and error that fuels innovation. AI may offer quick fixes, but it can’t replicate the creative process that arises from human ingenuity.\nDigital Dependency\ndigital dependency'

In [12]:
## Retriever

retriever = db.as_retriever()
docs = retriever.invoke(query)
docs[0].page_content

'One of the most significant impacts of the ChatGPT Effect is the potential decline in critical thinking skills. When we depend on AI to think for us, we lose the ability to evaluate information critically. AI tools provide answers, but they don’t teach us how to question, doubt, or verify. This lack of critical engagement can make us more susceptible to misinformation and reduce our ability to form well-rounded opinions.\n\nCreativity thrives on challenges, mistakes, and the journey of discovery. By outsourcing our creative tasks to AI, we risk losing the ability to think outside the box. Writing an essay, composing music, or solving a complex problem involves a series of trial and error that fuels innovation. AI may offer quick fixes, but it can’t replicate the creative process that arises from human ingenuity.\nDigital Dependency\ndigital dependency'

In [13]:
## Similarity Search With Score

docs_and_score = db.similarity_search_with_score(query)

docs_and_score

[(Document(metadata={'source': 'test.txt'}, page_content='One of the most significant impacts of the ChatGPT Effect is the potential decline in critical thinking skills. When we depend on AI to think for us, we lose the ability to evaluate information critically. AI tools provide answers, but they don’t teach us how to question, doubt, or verify. This lack of critical engagement can make us more susceptible to misinformation and reduce our ability to form well-rounded opinions.\n\nCreativity thrives on challenges, mistakes, and the journey of discovery. By outsourcing our creative tasks to AI, we risk losing the ability to think outside the box. Writing an essay, composing music, or solving a complex problem involves a series of trial and error that fuels innovation. AI may offer quick fixes, but it can’t replicate the creative process that arises from human ingenuity.\nDigital Dependency\ndigital dependency'),
  5272.4277),
 (Document(metadata={'source': 'test.txt'}, page_content='In 

In [15]:
# passing vectors as query

embedding_vector =  embeddings.embed_query(query)
embedding_vector

[-0.5206431150436401,
 0.12461890280246735,
 -2.130985736846924,
 0.4654954671859741,
 1.0467031002044678,
 1.4107974767684937,
 -3.2603065967559814,
 -1.2861350774765015,
 -1.079707384109497,
 0.684618353843689,
 -2.2412972450256348,
 -1.096006155014038,
 -2.3924484252929688,
 -0.8956090807914734,
 0.2857123911380768,
 0.6047345399856567,
 0.02842935547232628,
 0.3521821200847626,
 -7.846705436706543,
 -0.4928514361381531,
 1.3318560123443604,
 -0.7841024398803711,
 -0.3846088945865631,
 -2.8366565704345703,
 0.019411839544773102,
 -2.149183988571167,
 1.5696191787719727,
 -1.1300100088119507,
 -0.09010498225688934,
 -0.31162238121032715,
 -1.1074082851409912,
 -1.872164011001587,
 -0.5083755850791931,
 -0.33001986145973206,
 1.067785382270813,
 2.0208466053009033,
 -1.2742568254470825,
 -1.3753875494003296,
 1.5208672285079956,
 -1.4439926147460938,
 0.7669197916984558,
 0.6294550895690918,
 0.5595628023147583,
 -0.1620977222919464,
 0.521317720413208,
 0.8526819348335266,
 0.0215311

In [17]:
docs_score = db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(metadata={'source': 'test.txt'}, page_content='One of the most significant impacts of the ChatGPT Effect is the potential decline in critical thinking skills. When we depend on AI to think for us, we lose the ability to evaluate information critically. AI tools provide answers, but they don’t teach us how to question, doubt, or verify. This lack of critical engagement can make us more susceptible to misinformation and reduce our ability to form well-rounded opinions.\n\nCreativity thrives on challenges, mistakes, and the journey of discovery. By outsourcing our creative tasks to AI, we risk losing the ability to think outside the box. Writing an essay, composing music, or solving a complex problem involves a series of trial and error that fuels innovation. AI may offer quick fixes, but it can’t replicate the creative process that arises from human ingenuity.\nDigital Dependency\ndigital dependency'),
 Document(metadata={'source': 'test.txt'}, page_content='In an era where ins

### Saving to local and loading

In [18]:
db.save_local("faiss_index")

In [20]:
#loadnig the index
new_db = FAISS.load_local("faiss_index",embeddings, allow_dangerous_deserialization=True)
docs = new_db.similarity_search(query)

In [21]:
docs

[Document(metadata={'source': 'test.txt'}, page_content='One of the most significant impacts of the ChatGPT Effect is the potential decline in critical thinking skills. When we depend on AI to think for us, we lose the ability to evaluate information critically. AI tools provide answers, but they don’t teach us how to question, doubt, or verify. This lack of critical engagement can make us more susceptible to misinformation and reduce our ability to form well-rounded opinions.\n\nCreativity thrives on challenges, mistakes, and the journey of discovery. By outsourcing our creative tasks to AI, we risk losing the ability to think outside the box. Writing an essay, composing music, or solving a complex problem involves a series of trial and error that fuels innovation. AI may offer quick fixes, but it can’t replicate the creative process that arises from human ingenuity.\nDigital Dependency\ndigital dependency'),
 Document(metadata={'source': 'test.txt'}, page_content='In an era where ins